In [83]:
import numpy as np
from scipy.spatial import Delaunay
from astropy.table import Table as t, vstack
from collections import defaultdict
import os
from astropy.io import fits

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.dpi'] = 360
matplotlib.rcParams['text.usetex'] = True
os.environ['PATH'] = '/Library/TeX/texbin:' + os.environ['PATH']
from matplotlib.ticker import MaxNLocator, AutoMinorLocator

plt.rcParams.update({
    'font.family': 'serif',
    'font.size': 12,
    'axes.labelsize': 12,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
})
plt.style.use('dark_background')
cmap = sns.color_palette('mako')#, as_cmap=True)
cmap

[(0.18195582, 0.11955283, 0.23136943),
 (0.25307401, 0.23772973, 0.48316271),
 (0.21607792, 0.39736958, 0.61948028),
 (0.20344718, 0.56074869, 0.65649508),
 (0.25187832, 0.71827158, 0.67872193),
 (0.54578602, 0.8544913, 0.69848331)]

# EDR

### North

In [ ]:
base_dir = './data/edr/vac/edr/lss/v2.0/LSScats/clustering/'
dat = f'{base_dir}ELG_N_clustering.dat.fits'

In [ ]:
data = t.read(dat, hdu=1)

tables_n = []
for i in range(10):
    tab = t.read(f'{base_dir}ELG_N_{i}_clustering.ran.fits', hdu='LSS')
    tables_n.append(tab)

rand = tab#vstack(tables_n)

In [ ]:
list(np.unique(data['ROSETTE_NUMBER']))

#### Filter by Z

In [ ]:
cuts_n = [(0.93,0.96), (0.96,1.0), (0.83,0.9), (0.86,0.9), (0.96,1.0),
        (0.86,0.9), (0.83,0.86), (0.96,1.0), (0.86,0.9), (0.93,0.96)]
rn, i = 7, 2

In [ ]:
mask_d = (data['ROSETTE_NUMBER'] == rn) & (data['Z'] > cuts_n[i][0]) & (data['Z'] < cuts_n[i][1])
data_d = data[mask_d]['TARGETID', 'Z', 'RA', 'DEC']

mask_r = (rand['ROSETTE_NUMBER'] == rn) & (rand['Z'] > cuts_n[i][0]) & (rand['Z'] < cuts_n[i][1])
data_r = rand[mask_r]['TARGETID', 'Z', 'RA', 'DEC']

data_t = vstack([data_d, data_r])
np.shape(data_t)

#### Triangulation

In [ ]:
tri_data = np.vstack([data_t['RA'], data_t['DEC']]).T
tri = Delaunay(tri_data)

In [ ]:
fig, ax = plt.subplots()
ax.grid(linewidth=0.2, zorder=0)

ax.triplot(tri_data[:,0], tri_data[:,1], tri.simplices, linewidth=0.2, color='k', zorder=1)
ax.scatter(data_r['RA'], data_r['DEC'], label='Data', s=0.5, c=[cmap[3]], zorder=3)
ax.scatter(data_d['RA'], data_d['DEC'], label='Data', s=0.5, c=[cmap[0]], zorder=5)

ax.legend(loc='lower left', fontsize=8)
ax.set_title(f'Rosette {rn}\n'+f'{cuts_n[i][0]} '+'$< Z <$' + f'{cuts_n[i][1]}',
             fontsize=10, y=1.01)

ax.set_xlabel('RA [deg]')
ax.set_ylabel('DEC [deg]')
ax.set_box_aspect(1)
plt.show()

In [ ]:
n_real = len(data_d)
n_total = len(data_t)
is_real = np.zeros(n_total, dtype=bool)
is_real[:n_real] = True

neighbors = defaultdict(set)
for simplex in tri.simplices:
    i, j, k = simplex
    neighbors[i].update([j, k])
    neighbors[j].update([i, k])
    neighbors[k].update([i, j])

connections = {}
for i in range(n_total):
    nbrs = neighbors[i]
    real_count = sum(is_real[j] for j in nbrs)
    rand_count = len(nbrs) - real_count
    connections[i] = {'real': real_count, 'random': rand_count}

#### Classify

In [ ]:
for i in range(n_total):
    real = connections[i]['real']
    rand_ = connections[i]['random']
    r = (real-rand_)/(real+rand_)
    if -1.0 <= r and r <= -0.9:
        type = 'void'
    elif -0.9 < r and r <= 0.0:
        type = 'sheet'
    elif 0.0 < r and r <= 0.9:
        type = 'filament'
    elif 0.9 < r and r <= 1.0:
        type = 'knot'
    else:
        print(f'Error: r={r}')

    connections[i]['r'] = r
    connections[i]['type'] = type

In [ ]:
voids, sheets, filaments, knots = [], [], [], []
for key, value in connections.items():
    data_t = tri_data[key]
    if value['type'] == 'void':
        voids.append(data_t)
    elif value['type'] == 'sheet':
        sheets.append(data_t)
    elif value['type'] == 'filament':
        filaments.append(data_t)
    elif value['type'] == 'knot':
        knots.append(data_t)
    else:
        print(f'Error: {value["type"]}')

In [ ]:
len(voids), len(sheets), len(filaments), len(knots)

#### Loop for all rosettes

In [ ]:
rosettes = list(np.unique(data['ROSETTE_NUMBER']))
for i, rosette in enumerate(rosettes):
    mask_d = (data['ROSETTE_NUMBER'] == rosette) & (data['Z'] > cuts_n[i][0]) & (data['Z'] < cuts_n[i][1])
    data_d = data[mask_d]['TARGETID', 'Z', 'RA', 'DEC']

    mask_r = (rand['ROSETTE_NUMBER'] == rosette) & (rand['Z'] > cuts_n[i][0]) & (rand['Z'] < cuts_n[i][1])
    data_r = rand[mask_r]['TARGETID', 'Z', 'RA', 'DEC']

    data_t = vstack([data_d, data_r])
    tri_data = np.vstack([data_t['RA'], data_t['DEC']]).T
    tri = Delaunay(tri_data)

    n_real = len(data_d)
    n_total = len(data_t)
    is_real = np.zeros(n_total, dtype=bool)
    is_real[:n_real] = True

    neighbors = defaultdict(set)
    for simplex in tri.simplices:
        i, j, k = simplex
        neighbors[i].update([j, k])
        neighbors[j].update([i, k])
        neighbors[k].update([i, j])

    connections = {}
    for i in range(n_total):
        nbrs = neighbors[i]
        real_count = sum(is_real[j] for j in nbrs)
        rand_count = len(nbrs) - real_count
        connections[i] = {'real': real_count, 'random': rand_count}

    for i in range(n_total):
        real = connections[i]['real']
        random = connections[i]['random']
        r = (real-random)/(real+random)
        if -1.0 <= r and r <= -0.9:
            type = 'void'
        elif -0.9 < r and r <= 0.0:
            type = 'sheet'
        elif 0.0 < r and r <= 0.9:
            type = 'filament'
        elif 0.9 < r and r <= 1.0:
            type = 'knot'
        else:
            print(f'Error: r={r}')

        connections[i]['r'] = r
        connections[i]['type'] = type

    voids, sheets, filaments, knots = [], [], [], []
    for key, value in connections.items():
        data_t = tri_data[key]
        if value['type'] == 'void':
            voids.append(data_t)
        elif value['type'] == 'sheet':
            sheets.append(data_t)
        elif value['type'] == 'filament':
            filaments.append(data_t)
        elif value['type'] == 'knot':
            knots.append(data_t)
        else:
            print(f'Error: {value["type"]}')

    print(f'rosette {rosette}, voids:{len(voids)}, sheets:{len(sheets)}, filaments:{len(filaments)}, knots:{len(knots)}')

In [ ]:
def stack_randoms(data_path, n_r):
    data = t.read(data_path, hdu=1)
    tables_n = []
    for i in range(n_r):
        tab = t.read(f'{base_dir}ELG_N_{i}_clustering.ran.fits', hdu='LSS')
        tables_n.append(tab)
    return tab#vstack(tables_n)

def mask_data(data, rand, rn, cuts_n, i):
    mask_d = (data['ROSETTE_NUMBER'] == rn) & (data['Z'] > cuts_n[i][0]) & (data['Z'] < cuts_n[i][1])
    data_d = data[mask_d]['TARGETID', 'Z', 'RA', 'DEC']

    mask_r = (rand['ROSETTE_NUMBER'] == rn) & (rand['Z'] > cuts_n[i][0]) & (rand['Z'] < cuts_n[i][1])
    data_r = rand[mask_r]['TARGETID', 'Z', 'RA', 'DEC']

    return data_d, vstack([data_d, data_r])

def get_r(data_d, data_t, rn, cuts_n, i):
    n_real = len(data_d)
    n_total = len(data_t)
    is_real = np.zeros(n_total, dtype=bool)
    is_real[:n_real] = True

    neighbors = defaultdict(set)
    for simplex in tri.simplices:
        i, j, k = simplex
        neighbors[i].update([j, k])
        neighbors[j].update([i, k])
        neighbors[k].update([i, j])

    connections = {}
    for i in range(n_total):
        nbrs = neighbors[i]
        real_count = sum(is_real[j] for j in nbrs)
        rand_count = len(nbrs) - real_count
        connections[i] = {'real': real_count, 'random': rand_count}
    return connections

In [ ]:
def classify_connections(connections, n_total):
    for i in range(n_total):
        real = connections[i]['real']
        random = connections[i]['random']
        r = (real-random)/(real+random)
        if -1.0 <= r and r <= -0.9:
            type = 'void'
        elif -0.9 < r and r <= 0.0:
            type = 'sheet'
        elif 0.0 < r and r <= 0.9:
            type = 'filament'
        elif 0.9 < r and r <= 1.0:
            type = 'knot'
        else:
            type = 'unknown'
            print(f'Error: r={r}')

        connections[i]['r'] = r
        connections[i]['type'] = type
    return connections

def classify_data(data_t, tri_data, connections):
    voids, sheets, filaments, knots = [], [], [], []

    for key, value in connections.items():
        data_t = tri_data[key]
        if value['type'] == 'void':
            voids.append(data_t)
        elif value['type'] == 'sheet':
            sheets.append(data_t)
        elif value['type'] == 'filament':
            filaments.append(data_t)
        elif value['type'] == 'knot':
            knots.append(data_t)
        else:
            print(f'Error: {value["type"]}')
    return voids, sheets, filaments, knots

In [ ]:
rosettes = list(np.unique(data['ROSETTE_NUMBER']))
connections

rand = stack_randoms(dat, len(rosettes))

for i, rosette in enumerate(rosettes):
    data_d, data_t = mask_data(data, rand, rosette, cuts_n, i)
    tri_data = np.vstack([data_t['RA'], data_t['DEC']]).T
    tri = Delaunay(tri_data)

    connections = get_r(data_d, data_t, rosette, cuts_n, i)
    connections_ = classify_connections(connections, len(data_t))

    voids, sheets, filaments, knots = classify_data(data_t, tri_data, connections_)
    print(f'rosette {rosette}, voids:{len(voids)}, sheets:{len(sheets)}, filaments:{len(filaments)}, knots:{len(knots)}')

In [ ]:
i, rosette = 6, 14
data_d, data_t = mask_data(data, rand, rosette, cuts_n, i)
tri_data = np.vstack([data_t['RA'], data_t['DEC']]).T
tri = Delaunay(tri_data)

connections = get_r(data_d, data_t, rosette, cuts_n, i)
connections_ = classify_connections(connections, len(data_t))

voids, sheets, filaments, knots = classify_data(data_t, tri_data, connections_)
print(f'rosette {rosette}, voids:{len(voids)}, sheets:{len(sheets)}, filaments:{len(filaments)}, knots:{len(knots)}')

#### Plot triangulation and types

In [ ]:
types = ['void', 'sheet', 'filament', 'knot']
is_type = {t: np.array([connections_[i]['type']==t+'s' or connections_[i]['type']==t
                        for i in range(len(tri_data))]) for t in types}

tri_by_type = {}
for t in types:
    mask = is_type[t]
    tri_by_type[t] = [simp for simp in tri.simplices
                      if all(mask[v] for v in simp)]

fig, axes = plt.subplots(1, 4, figsize=(16,4), sharey=True)

for ax, t in zip(axes, types):
    mask = is_type[t]
    tris = tri_by_type[t]

    for i0, i1, i2 in tris:
        xs = [tri_data[i0,0], tri_data[i1,0], tri_data[i2,0], tri_data[i0,0]]
        ys = [tri_data[i0,1], tri_data[i1,1], tri_data[i2,1], tri_data[i0,1]]
        ax.plot(xs, ys, '-', color='black', alpha=0.7, linewidth=0.5, zorder=0)

    ax.scatter(tri_data[mask,0], tri_data[mask,1],
               s=4, zorder=10, c=[cmap[types.index(t)]], edgecolors='black', linewidth=0.03)

    # ax.set_xlim(251, 253); ax.set_ylim(33.5, 35.5)
    ax.set_title(f'{t.capitalize()}s')
    ax.set_box_aspect(1)
    ax.set_xlabel('RA [deg]')

axes[0].set_ylabel('DEC [deg]')

plt.suptitle(f'Rosette {rosette}\n'+f'{cuts_n[i][0]} '+'$< Z <$' + f'{cuts_n[i][1]}', y=1.01)
# plt.savefig(f'rosette_{rosette}.png', dpi=360)
plt.show()

In [ ]:
n_real = len(data_d)
n_total = len(data_t)
is_real = np.zeros(n_total, dtype=bool)
is_real[:n_real] = True

types = ['void', 'sheet', 'filament', 'knot']
is_type = {
    t: np.array([connections_[i]['type'] == t for i in range(n_total)])
           for t in types
           }

tri_by_type = {
    t: [simp for simp in tri.simplices if all(is_type[t][v] for v in simp)]
    for t in types
}

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(16,4), sharey=True)

for ax, t in zip(axes, types):
    mask_t = is_type[t]
    tris = tri_by_type[t]
    mask_real = mask_t & is_real
    mask_rand = mask_t & ~is_real

    for i0, i1, i2 in tris:
        xs = [tri_data[i0,0], tri_data[i1,0], tri_data[i2,0], tri_data[i0,0]]
        ys = [tri_data[i0,1], tri_data[i1,1], tri_data[i2,1], tri_data[i0,1]]
        ax.plot(xs, ys, '-', color='black', alpha=0.7, linewidth=0.2, zorder=0)

    #  randomm
    ax.scatter(tri_data[mask_rand,0], tri_data[mask_rand,1],
               s=0.2, c='tab:blue', label='random', zorder=5)

    #  reall
    ax.scatter(tri_data[mask_real,0], tri_data[mask_real,1],
               s=5, c='navy', edgecolors='black', linewidth=0.03,
               label='real', zorder=10)

    ax.set_xlim(251, 253)
    ax.set_ylim(33.5, 35.5)
    ax.set_title(f'{t.capitalize()}s')
    ax.legend(loc='lower left', fontsize=8)
    ax.set_box_aspect(1)
    ax.set_xlabel('RA [deg]')

axes[0].set_ylabel('DEC [deg]')

handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center', ncol=2, frameon=False)

plt.suptitle(
    f'Rosette {rosette}\n' + f'{cuts_n[i][0]}' + r'$ < Z < $' + f'{cuts_n[i][1]}',
    y=1.02
)
# plt.tight_layout()
plt.show()

In [ ]:
for i, rosette in enumerate(rosettes):
    data_d, data_t = mask_data(data, rand, rosette, cuts_n, i)
    tri_data = np.vstack([data_t['RA'], data_t['DEC']]).T
    tri = Delaunay(tri_data)

    connections = get_r(data_d, data_t, rosette, cuts_n, i)
    connections_ = classify_connections(connections, len(data_t))

    voids, sheets, filaments, knots = classify_data(data_t, tri_data, connections_)
    print(f'rosette {rosette}, voids:{len(voids)}, sheets:{len(sheets)}, filaments:{len(filaments)}, knots:{len(knots)}')

    n_real = len(data_d)
    n_total = len(data_t)
    is_real = np.zeros(n_total, dtype=bool)
    is_real[:n_real] = True

    types = ['void', 'sheet', 'filament', 'knot']
    is_type = {
        t: np.array([connections_[i]['type'] == t for i in range(n_total)])
            for t in types
            }

    tri_by_type = {
        t: [simp for simp in tri.simplices if all(is_type[t][v] for v in simp)]
        for t in types
    }
    #!---------------------------------------------------------------------------------------
    fig, axes = plt.subplots(1, 4, figsize=(14,4), sharey=True)

    for ax, tp in zip(axes, types):
        mask_t = is_type[tp]
        mask_real = mask_t & is_real
        mask_rand = mask_t & ~is_real

        tris = [simp for simp in tri.simplices if all(mask_t[v] for v in simp)]

        # dif
        for simplex in tris:
            for i in range(3):
                i0, i1 = simplex[i], simplex[(i+1)%3]
                xs = [tri_data[i0,0], tri_data[i1,0]]
                ys = [tri_data[i0,1], tri_data[i1,1]]

                if is_real[i0] and is_real[i1]:
                    # both real
                    ax.plot(xs, ys, '-', color='navy', alpha=0.9, linewidth=0.7, zorder=10)
                else:
                    # at least one random
                    ax.plot(xs, ys, '--', color='tab:blue', alpha=0.7, linewidth=0.2, zorder=0)

        ax.scatter(tri_data[mask_rand,0], tri_data[mask_rand,1],
                s=3, c='tab:blue', edgecolors='none', zorder=5, label='random')

        ax.scatter(tri_data[mask_real,0], tri_data[mask_real,1],
                s=6, c='navy', edgecolors='black', linewidth=0.3,
                zorder=10, label='real')

        ax.set_title(f'{tp.capitalize()}s')
        ax.set_box_aspect(1)
        ax.set_xlabel('RA [deg]')

        x0, x1 = ax.get_xlim()
        y0, y1 = ax.get_ylim()
        cx = 0.5*(x0 + x1)
        cy = 0.5*(y0 + y1)
        ax.set_xlim(cx - 0.7, cx + 0.7)
        ax.set_ylim(cy - 0.7, cy + 0.7)

    axes[0].set_ylabel('DEC [deg]')

    # handles, labels = axes[0].get_legend_handles_labels()
    # fig.legend(handles, labels, loc='lower center', ncol=2, frameon=False) #! no se ve porque se sale del limite

    plt.suptitle(
        f'Rosette {rosette}\n' + f'{cuts_n[i][0]} ' + r'$ < Z < $' + f' {cuts_n[i][1]}',
        # y=1.01
    )
    plt.tight_layout()
    plt.savefig(f'./plots/rosette_{rosette}_N.png', dpi=360)
    plt.show()

### South

In [ ]:
def stack_randoms(data_path, n_r):
    data = t.read(data_path, hdu=1)
    tables_n = []
    for i in range(n_r):
        tab = t.read(f'{base_dir}ELG_S_{i}_clustering.ran.fits', hdu='LSS')
        tables_n.append(tab)
    return tab#vstack(tables_n)

In [ ]:
dat_s = f'{base_dir}ELG_S_clustering.dat.fits'
data_s = t.read(dat_s, hdu=1)

rosettes = list(np.unique(data_s['ROSETTE_NUMBER']))
rand_s = stack_randoms(dat_s, len(rosettes))

cuts_s = [(0.93,0.96), (0.93,0.96), (0.93,0.96), (0.96,1.0), (0.9,0.93),
        (0.83,0.86), (0.86,0.9), (0.96,1.0), (0.96,1.0), (0.83,0.86)]

In [ ]:
for i, rosette in enumerate(rosettes):
    data_d, data_t = mask_data(data_s, rand_s, rosette, cuts_s, i)
    tri_data = np.vstack([data_t['RA'], data_t['DEC']]).T
    tri = Delaunay(tri_data)

    connections = get_r(data_d, data_t, rosette, cuts_s, i)
    connections_ = classify_connections(connections, len(data_t))

    voids, sheets, filaments, knots = classify_data(data_t, tri_data, connections_)
    print(f'rosette {rosette}, voids:{len(voids)}, sheets:{len(sheets)}, filaments:{len(filaments)}, knots:{len(knots)}')

    n_real = len(data_d)
    n_total = len(data_t)
    is_real = np.zeros(n_total, dtype=bool)
    is_real[:n_real] = True

    types = ['void', 'sheet', 'filament', 'knot']
    is_type = {
        t: np.array([connections_[i]['type'] == t for i in range(n_total)])
            for t in types
            }

    tri_by_type = {
        t: [simp for simp in tri.simplices if all(is_type[t][v] for v in simp)]
        for t in types
    }
    #!---------------------------------------------------------------------------------------
    fig, axes = plt.subplots(1, 4, figsize=(14,4), sharey=True)

    for ax, tp in zip(axes, types):
        mask_t = is_type[tp]
        mask_real = mask_t & is_real
        mask_rand  = mask_t & ~is_real

        tris = [simp for simp in tri.simplices if all(mask_t[v] for v in simp)]

        # dif
        for simplex in tris:
            vertices = simplex
            xs = tri_data[vertices, 0]
            ys = tri_data[vertices, 1]
            xs = np.append(xs, xs[0])
            ys = np.append(ys, ys[0])

            if all(is_real[v] for v in vertices):
                ax.plot(xs, ys, '-', color='navy', alpha=0.9, linewidth=0.5, zorder=10)
            else:
                # pass
                ax.plot(xs, ys, '--', color='tab:blue', alpha=0.6, linewidth=0.2, zorder=0)

        ax.scatter(tri_data[mask_rand,0], tri_data[mask_rand,1],
                s=1, c='tab:blue', zorder=5, label='random')

        ax.scatter(tri_data[mask_real,0], tri_data[mask_real,1],
                s=3, c='navy', edgecolors='black', zorder=10, label='real')

        ax.set_title(f'{tp.capitalize()}s')
        ax.set_box_aspect(1)
        ax.set_xlabel('RA [deg]')

        x0, x1 = ax.get_xlim()
        y0, y1 = ax.get_ylim()
        cx = 0.5*(x0 + x1)
        cy = 0.5*(y0 + y1)
        ax.set_xlim(cx - 0.7, cx + 0.7)
        ax.set_ylim(cy - 0.7, cy + 0.7)

    axes[0].set_ylabel('DEC [deg]')

    # handles, labels = axes[0].get_legend_handles_labels()
    # fig.legend(handles, labels, loc='lower center', ncol=2, frameon=False) #! se sale de la grafica por los limites

    plt.suptitle(
        f'Rosette {rosette}\n' + f'{cuts_s[i][0]} ' + r'$ < Z < $' + f' {cuts_s[i][1]}',
        #y=1.01
    )
    plt.tight_layout()
    plt.savefig(f'./plots/rosette_{rosette}_S.png', dpi=360)
    plt.show()


In [ ]:
cmap1 = sns.color_palette('mako', as_cmap=True)

for i, rosette in enumerate(rosettes):
    data_d, data_t = mask_data(data_s, rand_s, rosette, cuts_s, i)
    tri_data = np.vstack([data_t['RA'], data_t['DEC']]).T
    tri = Delaunay(tri_data)

    connections = get_r(data_d, data_t, rosette, cuts_s, i)
    connections_ = classify_connections(connections, len(data_t))

    voids, sheets, filaments, knots = classify_data(data_t, tri_data, connections_)
    print(f'rosette {rosette}, voids:{len(voids)}, sheets:{len(sheets)}, filaments:{len(filaments)}, knots:{len(knots)}')

    n_real = len(data_d)
    n_total = len(data_t)
    is_real = np.zeros(n_total, dtype=bool)
    is_real[:n_real] = True

    types = ['void', 'sheet', 'filament', 'knot']
    is_type = {
        t: np.array([connections_[i]['type'] == t for i in range(n_total)])
            for t in types
            }

    tri_by_type = {
        t: [simp for simp in tri.simplices if all(is_type[t][v] for v in simp)]
        for t in types
    }
    #!---------------------------------------------------------------------------------------
    fig, axes = plt.subplots(1, 4, figsize=(14,4), sharey=True)

    for ax, tp in zip(axes, types):
        mask_t = is_type[tp]
        mask_real = mask_t & is_real
        mask_rand = mask_t & ~is_real

        tris = [simp for simp in tri.simplices if all(mask_t[v] for v in simp)]

        # dif
        for simplex in tris:
            vertices = simplex
            xs = tri_data[vertices, 0]
            ys = tri_data[vertices, 1]
            xs = np.append(xs, xs[0])
            ys = np.append(ys, ys[0])

            if all(is_real[v] for v in vertices):
                # pass
                ax.plot(xs, ys, '-', color=cmap1(0.95), alpha=1.0, linewidth=0.4, zorder=10)
            else:
                # pass
                ax.plot(xs, ys, '--', color=cmap1(0.4), alpha=0.7, linewidth=0.2, zorder=0)

        ax.scatter(tri_data[mask_rand,0], tri_data[mask_rand,1],
                s=0.1, c=[cmap1(0.4)], zorder=5, label='random')

        ax.scatter(tri_data[mask_real,0], tri_data[mask_real,1],
                s=0.6, c=[cmap1(0.95)], zorder=10, label='real')

        ax.set_title(f'{tp.capitalize()}s')
        ax.set_box_aspect(1)
        ax.set_xlabel('RA [deg]')

        x0, x1 = ax.get_xlim()
        y0, y1 = ax.get_ylim()
        cx = 0.5*(x0 + x1)
        cy = 0.5*(y0 + y1)
        ax.set_xlim(cx - 0.7, cx + 0.7)
        ax.set_ylim(cy - 0.7, cy + 0.7)

    axes[0].set_ylabel('DEC [deg]')

    # handles, labels = axes[0].get_legend_handles_labels()
    # fig.legend(handles, labels, loc='lower center', ncol=2, frameon=False) #! se sale de la grafica por los limites

    plt.suptitle(
        f'Rosette {rosette}\n' + f'{cuts_s[i][0]} ' + r'$ < Z < $' + f' {cuts_s[i][1]}',
        #y=1.01
    )
    plt.tight_layout()
    plt.savefig(f'./plots/rosette_{rosette}_S.png', dpi=360)
    plt.show()

In [ ]:
cmap1 = sns.color_palette('mako', as_cmap=True)
#plt.get_cmap('inferno')
n_types = len(types)+1

fig, ax = plt.subplots()

for idx, tp in enumerate(types):
    mask_t = is_type[tp]
    tris_t = [simp for simp in tri.simplices if all(mask_t[v] for v in simp)]

    fcolor = (idx / (n_types - 1))+0.2
    print(f'{tp} color: {fcolor}')
    col = cmap1(fcolor)

    for simplex in tris_t:
        for j in range(3):
            i0, i1 = simplex[j], simplex[(j+1)%3]
            xs, ys = tri_data[[i0,i1],0], tri_data[[i0,i1],1]
            if is_real[i0] and is_real[i1]:
                ax.plot(xs, ys, '-',  color=col, alpha=0.9, linewidth=0.5, zorder=1)
            else:
                ax.plot(xs, ys, '--', color=col, alpha=0.4, linewidth=0.3, zorder=0)

    ax.scatter(tri_data[mask_t & ~is_real,0],
               tri_data[mask_t & ~is_real,1],
               s=1, c=[col], alpha=0.4,
               zorder=2)

    ax.scatter(tri_data[mask_t & is_real,0],
               tri_data[mask_t & is_real,1],
               s=1, c=[col],
               label=f'{tp.capitalize()}s',
               zorder=10)

ax.set_xlabel('RA [deg]')
ax.set_ylabel('DEC [deg]')
ax.set_aspect('equal', 'box')
ax.set_title(f'Rosette {rosette} — {cuts_s[i][0]} < Z < {cuts_s[i][1]}')

cx, cy = np.mean(tri_data, axis=0)
ax.set_xlim(cx - 0.8, cx + 0.8)
ax.set_ylim(cy - 0.8, cy + 0.8)

# handles, labels = ax.get_legend_handles_labels()
ax.legend( loc='lower left', markerscale=4.0,)

plt.tight_layout()
plt.show()